## 6.5.1 基础知识

In [2]:
class Number:
    def __init__(self, start):
        self.data = start
    def __sub__(self, other):
        return Number(self.data -other)

In [3]:
x = Number(5)
y = x -2
print(y.data)

3


常见的运算符重载方法

![](.6_images/d1dd2740.png)

![](.6_images/6746e547.png)

## 6.5.2 索引和分片

In [4]:
class Indexer:
    def __getitem__(self, index):
        return index ** 2

In [5]:
x = Indexer()
print(x[2])

4


In [6]:
for i in range(5):
    print(x[i], end=' ')

0 1 4 9 16 

In [7]:
# getitem会被分片表达式调用
l = [5,6,7,8,9]
print(l[2:4])
# 分片实际上就是python提供的一个语法糖，实际是下面这样的
print(l[slice(2,4)])

[7, 8]
[7, 8]


In [8]:
class Indexer:
    data = [5,6,7,8,9]
    def __getitem__(self, index):
        print('get item', index)
        return self.data[index]

In [9]:
x = Indexer()
print(x[0])
print(x[1])
print(x[-1])
print(x[2:4])
print(x[:-1])
print(x[::2])

get item 0
5
get item 1
6
get item -1
9
get item slice(2, 4, None)
[7, 8]
get item slice(None, -1, None)
[5, 6, 7, 8]
get item slice(None, None, 2)
[5, 7, 9]


In [10]:
# 我们可以直接获取到我们传入的参数
class Indexer:
    def __getitem__(self, index):
        if isinstance(index, int):
            print('indexing', index)
        else:
            print('slicing', index.start, index.stop, index.step)

In [11]:
x = Indexer()
print(x[99])
print(x[1:99:2])

indexing 99
None
slicing 1 99 2
None


## 6.5.3 索引迭代

In [12]:
class StepperIndex:
    def __getitem__(self, i):
        return self.data[i]

In [13]:
x = StepperIndex()
x.data = 'spam'
print(x[1])

p


In [14]:
# 有了这个getItem后我们就可以使用for来迭代了
for item in x:
    print(item, end=' ')

s p a m 

In [15]:
# 任何支持for循环的类也会自动支持python所有的迭代上下文
print('p' in x)
print([c for c in x])
print(list(map(str.upper, x)))
(a,b,c,d) = x
print(a,b,c,d)
print(list(x),tuple(x),''.join(x))

True
['s', 'p', 'a', 'm']
['S', 'P', 'A', 'M']
s p a m
['s', 'p', 'a', 'm'] ('s', 'p', 'a', 'm') spam


## 6.5.4 可迭代对象

In [16]:
# 上面的getItem实际上只是没有找到对应方法时才会调用
# 实际上可以迭代的对象可以通过下面的方法来进行调用
class Squares:
    def __init__(self, start, stop):
        self.value = start-1
        self.stop = stop
    def __iter__(self):
        return self
    def __next__(self):
        if self.value == self.stop:
            raise StopIteration
        self.value += 1
        return self.value ** 2

In [17]:
for i in Squares(1,5):
    print(i, end=' ')

1 4 9 16 25 

In [18]:
# 上面这个其实就是一个迭代器
x = Squares(1,3)
i = iter(x)
print(next(i))
print(next(i))
print(next(i))
print(next(i))

1
4
9


StopIteration: 

In [19]:
# 实际上这个可迭代对象不支持index操作
x = Squares(1,5)
print(x[1])

TypeError: 'Squares' object is not subscriptable

In [20]:
# 类里面的迭代对象通常只能迭代一次
x = Squares(1,5)
print([n for n in x])
print([n for n in x])

[1, 4, 9, 16, 25]
[]


In [21]:
# 所有我们每次迭代都必须要创建一个新的迭代对象
print([n for n in Squares(1,5)])
print(list(Squares(1,3)))

[1, 4, 9, 16, 25]
[1, 4, 9]


In [23]:
# 我们也可以间接使用其他类来转换一下
x = Squares(1,5)
print(tuple(x), tuple(x))
x = list(Squares(1,5))
print(tuple(x), tuple(x))

(1, 4, 9, 16, 25) ()
(1, 4, 9, 16, 25) (1, 4, 9, 16, 25)


In [24]:
# 我们先复习一下yield的用法
def gen(x):
    for i in range(x): yield i**2
g = gen(5)
print(g.__iter__() == g)
i = iter(g)
print(next(i))
print(next(i))
print(list(gen(5)))

True
0
1
[0, 1, 4, 9, 16]


In [25]:
# 下面我们给类加上yield
class Squares:
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    def __iter__(self):
        for value in range(self.start, self.stop+1):
            yield value**2

In [26]:
# 用法如下
for i in Squares(1,5): print(i, end=' ')

1 4 9 16 25 

In [27]:
# 这个类同样可以使用生成器的一些方法
s = Squares(1,3)
print(s)
i = iter(s)
print(i)
print(next(i))
print(next(i))
print(next(i))
print(next(i))

<generator object Squares.__iter__ at 0x000001CE0DEDF0C8>
1
4
9


StopIteration: 

In [28]:
# 不一定需要重载，我们也可以自己写
class Squares:
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    def gen(self):
        for value in range(self.start, self.stop+1):
            yield value**2

In [29]:
for i in Squares(1,5).gen():
    print(i, end=' ')

1 4 9 16 25 

In [30]:
# 下面我们手动模拟一下这个迭代器
class SquaresIter:
    def __init__(self, start, stop):
        self.value = start -1
        self.stop = stop
    def __next__(self):
        if self.value == self.stop:
            raise StopIteration
        self.value += 1
        return self.value ** 2


class Squares:
    def __init__(self,start,stop):
        self.start = start
        self.stop = stop
    def __iter__(self):
        return SquaresIter(self.start, self.stop)

In [31]:
for i in Squares(1,5):
    print(i,end =' ')

1 4 9 16 25 

In [32]:
# 上面这个也支持那些基本的迭代操作
s = Squares(1,5)
i = iter(s)
print(next(i))
print(next(i))

1
4


## 6.5.5 成员关系

In [33]:
class Container:
    def __contains__(self, item):
        print("调用contains：", item)
        return True

In [34]:
x = Container()
print(4 in x)

调用contains： 4
True


## 6.5.6 属性访问

In [35]:
class Empty:
    def __getattr__(self, attr_name):
        if attr_name == 'age':
            return 40
        else:
            raise AttributeError(attr_name)

In [36]:
x = Empty()
print(x.age)
print(x.name)

40


AttributeError: name

In [37]:
class AccessControl:
    def __setattr__(self, key, value):
        if key == 'age':
            self.__dict__[key] = value+10
        else:
            raise AttributeError(key+'not allowed')

In [38]:
x = AccessControl()
x.age = 40
print(x.age)
x.name = 'bob'

50


AttributeError: namenot allowed

In [ ]:
class AccessControl:
    def __setattr__(self, key, value):
        if key == 'age':
            # 注意我们不能这样调用，要不然会触发无限递归循环
            self.age  = value+10
            # 这样也不行
            setattr(self,key,value+10)
        else:
            raise AttributeError(key+'not allowed')

In [39]:
# 我们可以利用上面这个来实现私有化调用的功能
# 我们想定义一个异常
class PrivateExc(Exception): pass
class Privacy:
    def __setattr__(self, key, value):
        if key in self.private:
            raise PrivateExc(key, value)
        else:
            self.__dict__[key] = value
class Test1(Privacy):
    private = ['age']
class Test2(Privacy):
    private = ['name','pay']
    def __init__(self):
        self.__dict__['name'] = 'tom'

In [40]:
x = Test1()
y = Test2()
x.name = 'bob'
print(x.name)
y.name = 'sue'

bob


PrivateExc: ('name', 'sue')

In [41]:
y.age = 30
print(y.age)
x.age = 40

30


PrivateExc: ('age', 40)

## 6.5.7 字符串显示

In [42]:
class adder:
    def __init__(self, value=0):
        self.data = value
    def __add__(self, other):
        self.data += other

In [43]:
x = adder()
print(x)

In [44]:
class adder:
    def __init__(self, value=0):
        self.data = value
    def __add__(self, other):
        self.data += other
    def __repr__(self):
        return "add repr(%s)" % self.data

In [45]:
x = adder(2)
x + 1
print(x)

add repr(3)


In [46]:
# 这两者返回的内容是一样的
print(str(x), repr(x))

add repr(3) add repr(3)


In [47]:
# 我们也可以使用str
class addstr(adder):
    def __str__(self):
        return '[value: %s]' % self.data

In [48]:
x = addstr(3)
x+1
print(x)
print(str(x), repr(x))

[value: 4]
[value: 4] add repr(4)


In [49]:
# 注意我们的str方法必须要返回一个字符串
class Printer:
    def __init__(self, val):
        self.val = val
    def __str__(self):
        return str(self.val)

In [50]:
# 建议使用repr来打印，而不是str
objs = [Printer(2), Printer(3)]
print(objs)
for x in objs:
    print(x)

[<__main__.Printer object at 0x000001CE0DE36A08>, <__main__.Printer object at 0x000001CE0DE36188>]
2
3


In [51]:
class Printer:
    def __init__(self, val):
        self.val = val
    def __repr__(self):
        return str(self.val)
objs = [Printer(2), Printer(3)]
print(objs)
for x in objs:
    print(x)

[2, 3]
2
3


## 6.5.9 右侧加法和原位置加法

In [52]:
class Adder:
    def __init__(self, value=0):
        self.data = value
    def __add__(self, other):
        return self.data + other

In [53]:
x = Adder(5)
# 上面这个支持左侧的加法
print(x+2)
# 但是不支持右侧加法
print(2+x)

7


TypeError: unsupported operand type(s) for +: 'int' and 'Adder'

In [54]:
# 我们可以这样
class Commuter1:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        # 注意下面这个顺序要反转一下，要不然会重复触发方法
        print('radd', self.val, other)
        return other+self.val

In [55]:
x = Commuter1(88)
y = Commuter1(99)
print(x+1)
print(1+y)
print(x+y)

add 88 1
89
radd 99 1
100
add 88 <__main__.Commuter1 object at 0x000001CE0DEFC5C8>
radd 99 88
187


In [56]:
# 当然，我们可以重用add方法
class Commuter2:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        return self.__add__(other)

In [57]:
x = Commuter2(88)
print(2+x)

add 88 2
90


In [58]:
# 也可以这样
class Commuter3:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    def __radd__(self, other):
        return self + other
x = Commuter3(88)
print(2+x)

add 88 2
90


In [59]:
# 或者这样
class Commuter4:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    __radd__ = __add__
x = Commuter3(88)
print(2+x)

add 88 2
90


In [60]:
class Commuter5:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        return Commuter5(self.val+other)
    def __radd__(self, other):
        return Commuter5(other + self.val)
x = Commuter5(88)
y = Commuter5(99)
print((x+10).val)
print((10+y).val)
# 如果两个类相加会直接嵌套
z = x+y
print(z.val)
print(z.val.val)

98
109
187


In [61]:
# 解决方法就是加一个判断
class Commuter5:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        if isinstance(other, Commuter5):
            other = other.val
        return Commuter5(self.val+other)
    def __radd__(self, other):
        return Commuter5(other + self.val)
# 这样就不会嵌套了
x = Commuter5(88)
y = Commuter5(99)
z = x+y
print(z.val)

187


In [62]:
# 下面我们来模拟原位置加法
class Number:
    def __init__(self,val):
        self.val = val
    def __iadd__(self, other):
        self.val += other
        return self

In [63]:
x = Number(5)
x += 1
x += 1
print(x.val)

7


In [64]:
# 如果是可变对象，我们初始化的时候需要这样初始化
y = Number([1])
y+=[2]
y+=[3]
print(y.val)

[1, 2, 3]


In [65]:
# 当然也可以使用常规的add方法
class Number:
    def __init__(self, val):
        self.val = val
    def __add__(self, other):
        return Number(self.val + other)

In [66]:
x = Number(5)
x+=1
x+=1
print(x.val)

7


## 6.5.10 调用表达式

In [67]:
class Callee:
    def __call__(self, *args, **kwargs):
        print('called:', args,kwargs)

In [68]:
c = Callee()
c(1,2,3)
c(1,2,3,x=4,y=5)

called: (1, 2, 3) {}
called: (1, 2, 3) {'x': 4, 'y': 5}


In [69]:
# 我们可以这样操作
class Prod:
    def __init__(self, value):
        self.value = value
    def __call__(self, other):
        return self.value * other
x = Prod(2)
print(x(3))
print(x(4))

6
8


## 6.5.11 比较运算

In [70]:
class C:
    data = 'spam'
    def __gt__(self, other):
        return self.data > other
    def __lt__(self, other):
        return self.data < other

In [71]:
x = C()
print(x > 'ham')
print(x < 'ham')

True
False


## 6.5.12 布尔测试

In [72]:
class Truth:
    def __bool__(self): return True
x = Truth()
if x: print('yes!')

yes!


In [73]:
class Truth:
    def __bool__(self): return False
x = Truth()
if not x: print('no!')

no!


In [74]:
# 长度也可以表示bool值
class Truth:
    def __len__(self): return 0
x = Truth()
if not x:
    print("no !")

no !


In [75]:
# 如果两者都有，那么优先选择bool
class Truth:
    def __bool__(self): return True
    def __len__(self): return 0
x = Truth()
if x:
    print("yes !")

yes !


In [76]:
# 如果都没有那么就是true
class Truth: pass
print(bool(Truth()))

True


## 6.5.13 对象构造函数

In [77]:
class Life:
    def __init__(self, name='unknown'):
        print('hello', name)
        self.name = name
    def live(self):
        print(self.name)
    def __del__(self):
        print('goodbye', self.name)

In [78]:
brain = Life('brain')
brain.live()
brain = 'loretta'

hello brain
brain
goodbye brain
